# PDE SDK Introduction

This notebook demonstrates the basic usage of the PDE SDK for solving partial differential equations.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pde_sdk.domains import UniformGrid1D
from pde_sdk.equations import HeatEquation1D
from pde_sdk.boundaries import DirichletBC
from pde_sdk.solvers import ExplicitEuler1D


## 1. Set up the Problem

We'll solve the 1D heat equation with Dirichlet boundary conditions.


In [ ]:
# Create grid
nx = 101
length = 1.0
grid = UniformGrid1D(nx=nx, length=length)

# Define initial condition
initial_condition = lambda x: np.sin(np.pi * x)

# Set up equation
alpha = 0.01  # Diffusion coefficient
equation = HeatEquation1D(
    alpha=alpha,
    grid=grid,
    left_bc=DirichletBC(0.0),
    right_bc=DirichletBC(0.0),
    initial_condition=initial_condition
)


## 2. Solve the Equation

Use explicit Euler method with progress tracking.


In [ ]:
# Create solver
dt = 1e-4  # Timestep
solver = ExplicitEuler1D(dt=dt)

# Solve
t_final = 0.1
solution = solver.solve(equation, t_final=t_final, verbosity='summary')


## 3. Visualize Results

Compare numerical solution with analytical solution.


In [ ]:
# Analytical solution
u_exact = np.exp(-np.pi**2 * alpha * t_final) * np.sin(np.pi * grid.x)

# Plot comparison
from pde_sdk.visualization import plot_comparison, plot_error

plot_comparison(grid.x, solution, u_exact, title="1D Heat Equation Solution")
plt.show()

# Plot error
error = np.abs(solution - u_exact)
plot_error(grid.x, error, log_scale=True, title="Error Analysis")
plt.show()

print(f"Maximum error: {np.max(error):.2e}")


## 4. Using the Configuration System

Automatically determine optimal parameters based on accuracy requirements.


In [ ]:
from pde_sdk.config import SolverConfig
import pde_sdk.solvers as solvers

# Create configuration
config = SolverConfig(
    target_accuracy=0.001,
    problem_type='heat_1d',
    alpha=0.01,
    domain_length=1.0
)

print(f"Recommended nx: {config.nx}")
print(f"Recommended dt: {config.dt:.2e}")
print(f"Recommended solver: {config.solver_type}")

# Use configuration
grid_config = UniformGrid1D(**config.get_grid_params())
solver_class = getattr(solvers, config.solver_type)
solver_config = solver_class(**config.get_solver_params())

# Set up and solve
equation_config = HeatEquation1D(
    alpha=config.alpha,
    grid=grid_config,
    left_bc=DirichletBC(0.0),
    right_bc=DirichletBC(0.0),
    initial_condition=initial_condition
)

solution_config = solver_config.solve(equation_config, t_final=0.1, verbosity='summary')
